<a href="https://colab.research.google.com/github/antoreep-jana/YouTube_Code_Repositories/blob/main/Weekly%20Sessions/Weekly_Session__12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensorflow Model Sub-Classing

In [1]:
import tensorflow as tf

In [2]:
class CustomModel(tf.keras.Model):

  def __init__(self, num_classes = 1000, verbose = True, bn = True):

    super(CustomModel, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(32, (2,2), strides = 1, activation ='relu')
    self.max1 = tf.keras.layers.MaxPool2D((2,2))
    
    if bn:
      self.bn1 = tf.keras.layers.BatchNormalization()
    
      if verbose:
        print("Batch Normalization Layer addded!")
    else:
      if verbose:
        print("Batch Normalization Layer not added! :(")


    self.conv2 = tf.keras.layers.Conv2D(64, (2,2), strides = 1, activation = 'relu')
    self.max2 = tf.keras.layers.MaxPool2D((2,2))

    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(num_classes, activation = 'softmax') 


  def call(self, input_vector):
    
    x = self.conv1(input_vector)
    x = self.max1(x)
  
    
    x = self.bn1(x)
  
    x = self.conv2(x)
    x = self.max2(x)

    return self.dense(self.flatten(x))

  def fit(self):
    pass 
    

  def evaluate(self):
    pass 



  def summary(self):

    X = tf.keras.layers.Input(shape = (224,224,3))
    model = tf.keras.Model(inputs = [X], outputs = self.call(X))
    return model.summary()


In [3]:
model = CustomModel(100, verbose = True, bn = True)

Batch Normalization Layer addded!


In [4]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [5]:
model.build(input_shape = (None, 224,224,3))

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 223, 223, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 111, 111, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 64)      8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 193600)            0     

Custom Layers using TF Keras

Step1. Define the init() with layer parameter values <br>
Step2. Define the build() to build your layer with kernel matrix & bias<br>
Step3. Define the call() to define the forward pass in the layer. Usually matmul of inputs & weights (kernel) + bias

In [7]:
class CustomLayer(tf.keras.layers.Layer):

  def __init__(self, units = 128, activation = 'relu', initializer = 'he_normal', **kwargs):

    #self.name = name
    super(CustomLayer, self).__init__(**kwargs)
    self.units = units 
    self.activation = tf.keras.activations.relu
    self.initializer = tf.keras.initializers.HeNormal()



  def build(self, input_shape):
    
    self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer = self.initializer, trainable = True)
  

    self.b = self.add_weight(shape = (self.units,), initializer = self.initializer, trainable = True)


  def call(self, input_tensor):
    

    # WX + B
    result = tf.matmul(input_tensor, self.w) + self.b


    # act(WX + B)
    if self.activation:
      result = self.activation(result)

    return result

  def get_config(self):
   
    # Step 1 : Get the configuration from the base layer 

    # Step 2 : make a dictionary of what all you want to save and update

    # return the dictionary 

    config = super(CustomLayer, self).get_config()

    config.update({'units' : self.units, 'initializer' : self.initializer})

    return config



In [8]:
layer = CustomLayer()

In [9]:
ini = tf.random_uniform_initializer()
tensor = ini((4,3)) 

In [10]:
layer(tensor)

<tf.Tensor: shape=(4, 128), dtype=float32, numpy=
array([[0.        , 0.        , 0.03591833, 0.04578527, 0.        ,
        0.        , 0.00865745, 0.04825905, 0.09275782, 0.21675695,
        0.09951411, 0.04286058, 0.13201103, 0.        , 0.        ,
        0.        , 0.030543  , 0.24444295, 0.        , 0.0945041 ,
        0.11582466, 0.08580009, 0.        , 0.        , 0.05802542,
        0.        , 0.10579466, 0.01242325, 0.        , 0.        ,
        0.01142816, 0.17402491, 0.        , 0.13656592, 0.        ,
        0.06431761, 0.        , 0.        , 0.02273746, 0.00486709,
        0.        , 0.        , 0.07436603, 0.        , 0.        ,
        0.        , 0.02548445, 0.        , 0.        , 0.0516995 ,
        0.        , 0.        , 0.        , 0.12079151, 0.        ,
        0.21186809, 0.01811527, 0.12875393, 0.        , 0.        ,
        0.04945293, 0.1648202 , 0.07600613, 0.        , 0.        ,
        0.19022961, 0.15419747, 0.        , 0.        , 0.        

In [11]:
layer.weights

[<tf.Variable 'custom_layer/Variable:0' shape=(3, 128) dtype=float32, numpy=
 array([[-4.46873158e-01,  2.91794121e-01,  2.69454271e-01,
         -3.94226372e-01,  1.35026371e+00, -6.48073494e-01,
         -6.53121471e-01,  1.49925315e+00, -4.79453176e-01,
         -1.76100922e+00, -7.30481625e-01, -5.51447749e-01,
          4.47463304e-01,  1.16218996e+00,  3.32970768e-01,
          1.24046937e-01,  1.34260356e+00,  6.58862174e-01,
         -4.28647548e-01,  2.06697375e-01,  6.67816639e-01,
         -4.26777899e-01, -2.85686076e-01,  8.49508960e-03,
         -4.89140712e-02,  1.60511339e+00, -4.48267996e-01,
          1.00182533e+00, -6.78552747e-01, -6.20051563e-01,
          1.62535954e+00,  1.03039336e+00,  1.02727026e-01,
         -7.59870529e-01, -8.54778767e-01, -4.22056854e-01,
          4.16674286e-01, -1.72086284e-01,  2.84797810e-02,
         -1.26574516e+00,  6.70371413e-01,  1.05118382e+00,
         -2.46145755e-01, -1.20487824e-01,  9.60044861e-01,
          2.64593571e-0

In [12]:
layer.b

<tf.Variable 'custom_layer/Variable:0' shape=(128,) dtype=float32, numpy=
array([-0.02773117, -0.22504783,  0.04571313,  0.03565152, -0.00566988,
        0.00635459,  0.01546322,  0.08498171,  0.09122587,  0.20185424,
        0.10989417,  0.00279371,  0.17080843, -0.10122529,  0.005427  ,
       -0.18571088,  0.05353548,  0.23588228, -0.2780575 ,  0.10662503,
        0.13188027,  0.06273496, -0.11716955, -0.02317112,  0.04404229,
       -0.16982268,  0.06189066,  0.02379258, -0.17168063, -0.13582613,
        0.00664504,  0.14473793,  0.04109056,  0.13574323, -0.24587905,
        0.04810132, -0.03921025, -0.05516314,  0.02453772, -0.00461692,
       -0.04278491, -0.12248579,  0.07088179, -0.14170745, -0.04419668,
       -0.04832732,  0.01112884, -0.11870657, -0.02545032,  0.07407834,
       -0.04129823, -0.24364166, -0.05799844,  0.12294437, -0.2469328 ,
        0.2392126 ,  0.01100669,  0.10368402,  0.00995222, -0.13374467,
        0.05853174,  0.19989587,  0.09443251, -0.10665199, -0.

In [13]:
layer.get_config()

{'dtype': 'float32',
 'initializer': <keras.initializers.initializers_v2.HeNormal at 0x7f9ec0512710>,
 'name': 'custom_layer',
 'trainable': True,
 'units': 128}

In [14]:
layer1 = CustomLayer()

layer1.from_config(layer.get_config())

In [15]:
class NestedLayers(tf.keras.layers.Layer):

  def __init__(self, units = 128):

    self.layer1 = CustomLayer1()
    self.layer2 = CustomLayer2()
  
  def build():
    pass 


  def call():
    pass 



## Custom Loss Functions

In [16]:
class CustomLoss(tf.keras.losses.Loss):

  def __init__(self, y_pred, y_true):
    self.y_pred = y_pred
    self.y_true = y_true


  def call(self):
    result = tf.square(self.y_pred - self.y_true)

    return tf.reduce_mean(result, axis = -1)

In [17]:
loss1 = CustomLoss(tf.ones((2, 2,)), tf.zeros((2, 2)))

In [18]:
loss1.call()

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>

In [19]:
def mean_sq_loss(y_pred, y_true):

  result = tf.square(y_pred - y_true)

  return tf.reduce_mean(result, axis = -1)